In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
data = pd.read_csv('fifa21_train.csv')

In [3]:
data.shape

(11701, 101)

In [4]:
data.columns = data.columns.str.lower().str.replace(' ','_')
data.columns = data.columns.str.lower().str.replace('/','_')

In [5]:
data = data.rename(columns={'bp':'best_position',
                            'team_&_contract':'team_contract', 
                            'w_f':'weak_foot', 'sm': 'skill_moves', 'a_w': 'attacking_work_rate', 'd_w': 'defensive_work_rate', 'ir': 'international_reputation', 'wf': 'weak_foot' })

In [6]:
data.to_csv('fifa21_cleanedcolumnnsnames.csv')

## Checking

In [7]:
# data.head()
# data.columns.tolist()
# data1.dtypes.tolist()
# data.isna().count().tolist()
# data['height']

# Decisions.
- OVR = ATT + IR (Overal Rating equals attributes and international reputation).

- We know that the variables used to calculate OVR are the ones of the **attributes** of the player (crossing, dribbling, positioning, etc.) plus the **international reputation**.

- Our categorical column is **"best position"**.

- We also have to take into account how the OVR changes when the player does not play in its "best position".

- So we need to use the information of the **last columns as well** (the ones with the information of the players' performance in each postion).

- For these columns (the ones with "+"), we´re going to keep **the second number**. We think the first number can be calculated from the columns of attributes.

- We know some columns are the sum of other columns. We decided to keep just the columns of attributes that are not a summation of other columns.

- We cleaned and treated this data and obtained our first model.

# New DF.
Divide Columns, Choose Right

In [8]:
data_pos = data.loc[:, 'ls':'gk']

for col in data_pos.columns:
    data_pos[[col, col + '_right']] = data_pos[col].str.split('+', expand=True)

# Convertir las columnas resultantes en numéricas
data_pos = data_pos.astype(int)

data_pos = data_pos.drop(data_pos.loc[:, 'ls':'gk'].columns, axis=1)

Drop columns.

In [9]:
data_att = data.loc[:, 'crossing':'gk_reflexes']
data_att = data_att.drop(['skill', 'movement', 'power', 'mentality', 'power', 'mentality', 'defending', 'goalkeeping'], axis=1)

Clean star.

In [10]:
icon = '★'
data['international_reputation'] = data['international_reputation'].str.replace(icon, '')
data['international_reputation'] = data['international_reputation'].str.replace(' ', '')
data['international_reputation'] = data['international_reputation'].astype('int')

Our categorial feautre is column 'best position'. We encode it with one hot encoder.

In [11]:
data_cat = pd.DataFrame(data['best_position'])

encoder = OneHotEncoder(drop='first').fit(data_cat)
encoded = encoder.transform(data_cat).toarray()

data_cat = pd.DataFrame(encoded,columns=encoder.get_feature_names_out())

Clean NaN. Replace with mean value.

In [12]:
for col in data_att.columns:
    data_att[col] = data_att[col].fillna(data_att[col].mean())

In [13]:
# Check.
data_att.isna().sum().tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

Concatenate numerical.

In [14]:
data_num = pd.concat([data_att, data_pos, data['international_reputation']], axis=1)
# data_nor.isna().sum().tolist()

data_num.columns

Index(['crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'fk_accuracy', 'long_passing', 'ball_control',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'composure', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes', 'ls_right', 'st_right', 'rs_right', 'lw_right',
       'lf_right', 'cf_right', 'rf_right', 'rw_right', 'lam_right',
       'cam_right', 'ram_right', 'lm_right', 'lcm_right', 'cm_right',
       'rcm_right', 'rm_right', 'lwb_right', 'ldm_right', 'cdm_right',
       'rdm_right', 'rwb_right', 'lb_right', 'lcb_right', 'cb_right',
       'rcb_right', 'rb_right', 'gk_right', 'international_reputation'],
      dtype='object')

# Check correlation.

In [15]:
data1_target = pd.concat([data['ova'], data_num], axis=1)
data1_target.columns

Index(['ova', 'crossing', 'finishing', 'heading_accuracy', 'short_passing',
       'volleys', 'dribbling', 'curve', 'fk_accuracy', 'long_passing',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'composure', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes', 'ls_right', 'st_right', 'rs_right', 'lw_right',
       'lf_right', 'cf_right', 'rf_right', 'rw_right', 'lam_right',
       'cam_right', 'ram_right', 'lm_right', 'lcm_right', 'cm_right',
       'rcm_right', 'rm_right', 'lwb_right', 'ldm_right', 'cdm_right',
       'rdm_right', 'rwb_right', 'lb_right', 'lcb_right', 'cb_right',
       'rcb_right', 'rb_right', 'gk_right', 'international_reputation'],
      dtype='object')

In [16]:
correlation_matrix = data1_target.corr()
correlation_matrix.columns

Index(['ova', 'crossing', 'finishing', 'heading_accuracy', 'short_passing',
       'volleys', 'dribbling', 'curve', 'fk_accuracy', 'long_passing',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'composure', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes', 'ls_right', 'st_right', 'rs_right', 'lw_right',
       'lf_right', 'cf_right', 'rf_right', 'rw_right', 'lam_right',
       'cam_right', 'ram_right', 'lm_right', 'lcm_right', 'cm_right',
       'rcm_right', 'rm_right', 'lwb_right', 'ldm_right', 'cdm_right',
       'rdm_right', 'rwb_right', 'lb_right', 'lcb_right', 'cb_right',
       'rcb_right', 'rb_right', 'gk_right', 'international_reputation'],
      dtype='object')

We discard the features wihout a correlation larger than 0.2 with the target.

In [17]:
cor_ova = correlation_matrix[(correlation_matrix['ova']>0.2) | (correlation_matrix['ova']<(-0.2))]

cor_ova.T.columns

Index(['ova', 'crossing', 'finishing', 'heading_accuracy', 'short_passing',
       'volleys', 'dribbling', 'curve', 'fk_accuracy', 'long_passing',
       'ball_control', 'agility', 'reactions', 'shot_power', 'jumping',
       'stamina', 'strength', 'long_shots', 'aggression', 'interceptions',
       'positioning', 'vision', 'penalties', 'composure', 'marking',
       'standing_tackle', 'sliding_tackle', 'international_reputation'],
      dtype='object')

We select the columns that meet the criteria.

In [18]:
data_num = data_num[['crossing', 'finishing', 'heading_accuracy', 'short_passing',
       'volleys', 'dribbling', 'curve', 'fk_accuracy', 'long_passing',
       'ball_control', 'agility', 'reactions', 'shot_power', 'jumping',
       'stamina', 'strength', 'long_shots', 'aggression', 'interceptions',
       'positioning', 'vision', 'penalties', 'composure', 'marking',
       'standing_tackle', 'sliding_tackle', 'international_reputation']]
data_num

,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,...,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle,international_reputation
0,54,47,43,70,44.000000,61,44.000000,55,63,63,...,54,52.0,62.0,68.000000,54,54.000000,49,56,43.000000,1
1,66,79,76,68,76.000000,83,78.000000,72,63,79,...,54,33.0,78.0,72.000000,76,70.000000,35,20,22.000000,2
2,73,76,34,78,75.000000,85,89.000000,91,74,85,...,75,26.0,80.0,78.000000,73,82.000000,23,29,28.000000,2
3,44,42,58,62,36.000000,54,41.000000,46,57,61,...,61,57.0,31.0,54.000000,54,48.000000,55,58,55.000000,1
4,49,37,61,68,34.000000,64,44.000000,45,61,66,...,66,60.0,55.0,64.000000,49,58.000000,58,61,66.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,62,24,51,59,28.000000,57,49.000000,29,54,59,...,63,59.0,51.0,54.000000,39,46.000000,59,64,61.000000,1
11697,12,14,13,29,16.000000,17,15.000000,19,27,18,...,30,22.0,15.0,15.000000,19,56.000000,12,16,13.000000,1
11698,64,66,51,64,41.000000,71,57.000000,38,58,67,...,27,25.0,66.0,58.000000,65,58.000000,37,12,13.000000,1
11699,64,66,51,73,45.067324,74,49.617839,63,72,75,...,64,61.0,63.0,55.439194,53,59.947732,55,58,45.827702,3


# Normalize.

In [19]:
tra = MinMaxScaler().fit(data_num)
nor = tra.transform(data_num)

data_nor = pd.DataFrame(nor, columns = data_num.columns)

data_nor

,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,...,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle,international_reputation
0,0.545455,0.478261,0.431818,0.720930,0.465116,0.615385,0.444444,0.561798,0.642857,0.637363,...,0.517241,0.522222,0.638298,0.682353,0.534884,0.500000,0.516854,0.595238,0.440476,0.00
1,0.681818,0.826087,0.806818,0.697674,0.837209,0.857143,0.822222,0.752809,0.642857,0.813187,...,0.517241,0.311111,0.808511,0.729412,0.790698,0.690476,0.359551,0.166667,0.190476,0.25
2,0.761364,0.793478,0.329545,0.813953,0.825581,0.879121,0.944444,0.966292,0.773810,0.879121,...,0.758621,0.233333,0.829787,0.800000,0.755814,0.833333,0.224719,0.273810,0.261905,0.25
3,0.431818,0.423913,0.602273,0.627907,0.372093,0.538462,0.411111,0.460674,0.571429,0.615385,...,0.597701,0.577778,0.308511,0.517647,0.534884,0.428571,0.584270,0.619048,0.583333,0.00
4,0.488636,0.369565,0.636364,0.697674,0.348837,0.648352,0.444444,0.449438,0.619048,0.670330,...,0.655172,0.611111,0.563830,0.635294,0.476744,0.547619,0.617978,0.654762,0.714286,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,0.636364,0.228261,0.522727,0.593023,0.279070,0.571429,0.500000,0.269663,0.535714,0.593407,...,0.620690,0.600000,0.521277,0.517647,0.360465,0.404762,0.629213,0.690476,0.654762,0.00
11697,0.068182,0.119565,0.090909,0.244186,0.139535,0.131868,0.122222,0.157303,0.214286,0.142857,...,0.241379,0.188889,0.138298,0.058824,0.127907,0.523810,0.101124,0.119048,0.083333,0.00
11698,0.659091,0.684783,0.522727,0.651163,0.430233,0.725275,0.588889,0.370787,0.583333,0.681319,...,0.206897,0.222222,0.680851,0.564706,0.662791,0.547619,0.382022,0.071429,0.083333,0.00
11699,0.659091,0.684783,0.522727,0.755814,0.477527,0.758242,0.506865,0.651685,0.750000,0.769231,...,0.632184,0.622222,0.648936,0.534579,0.523256,0.570806,0.584270,0.619048,0.474139,0.50


# Concatenate.

In [20]:
data1 = pd.concat([data_cat, data_nor], axis=1)
# data1.describe().T
# data1.dtypes

# Train-Test Split.

In [21]:
X = data1
y = data['ova']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Linear Regression.

In [22]:
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

# R2.

In [23]:
predictions_test = lm.predict(X_test)

In [24]:
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

0.8791785791464536

In [25]:
r2_score(y_test, predictions_test)

0.8683942542061293

# MAE.

In [26]:
mae = mean_absolute_error(y_test, predictions_test)
mae

1.84929078640372

# MSE.

In [27]:
mse=mean_squared_error(y_test,predictions_test)
mse

5.963620514621829

# RMSE.

In [28]:
rmse = np.sqrt(mse)
rmse

2.44205252085655

# Create a DEF Function.

In [29]:
def clean (df):

    # Rename columns
    df.columns = df.columns.str.lower().str.replace(' ','_')
    df.columns = df.columns.str.lower().str.replace('/','_')
    df = df.rename(columns={'bp':'best_position',
                                'team_&_contract':'team_contract', 
                                'w_f':'weak_foot', 'sm': 'skill_moves', 'a_w': 'attacking_work_rate', 'd_w': 'defensive_work_rate', 'ir': 'international_reputation', 'wf': 'weak_foot' })
    # Edit positions
    data_pos = df.loc[:, 'ls':'gk']
    for col in data_pos.columns:
        data_pos[[col, col + '_right']] = data_pos[col].str.split('+', expand=True)
    data_pos = data_pos.astype(int)
    data_pos = data_pos.drop(data_pos.loc[:, 'ls':'gk'].columns, axis=1)
    
    # Edit attributes
    data_att = df.loc[:, 'crossing':'gk_reflexes']
    data_att = data_att.drop(['skill', 'movement', 'power', 'mentality', 'defending', 'goalkeeping'], axis=1)
    
    # Edit international reputation
    icon = '★'
    df['international_reputation'] = df['international_reputation'].str.replace(icon, '')
    df['international_reputation'] = df['international_reputation'].str.replace(' ', '')
    df['international_reputation'].astype('int')
    
    # Edit best position
    data_cat = pd.DataFrame(df['best_position'])
    encoder = OneHotEncoder(drop='first').fit(data_cat)
    encoded = encoder.transform(data_cat).toarray()
    data_cat = pd.DataFrame(encoded,columns=encoder.get_feature_names_out())
    
    # Fill na
    for col in data_att.columns:
        data_att[col] = data_att[col].fillna(data_att[col].mean())
    
    # Concat
    data_num = pd.concat([data_att, data_pos, df['international_reputation']], axis=1)
    
    # Choose num columns
    data_num = data_num[['crossing', 'finishing', 'heading_accuracy', 'short_passing',
           'volleys', 'dribbling', 'curve', 'fk_accuracy', 'long_passing',
           'ball_control', 'agility', 'reactions', 'shot_power', 'jumping',
           'stamina', 'strength', 'long_shots', 'aggression', 'interceptions',
           'positioning', 'vision', 'penalties', 'composure', 'marking',
           'standing_tackle', 'sliding_tackle']]
    
    # Normalise
    tra = MinMaxScaler().fit(data_num)
    nor = tra.transform(data_num)
    data_nor = pd.DataFrame(nor, columns = data_num.columns)
    
    # concatenate final
    df = pd.concat([data_cat, data_nor], axis=1)
    
    return df